In [1]:
!nvidia-smi

Sat Oct 28 03:28:29 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import xml.etree.ElementTree as xet
from glob import glob
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img , img_to_array
from tensorflow.keras.applications import MobileNetV2, InceptionV3 , InceptionResNetV2
from tensorflow.keras.layers import Dense , Flatten , Dropout ,Input
from tensorflow.keras.models import Model
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard

In [5]:
df = pd.read_csv("/content/drive/MyDrive/new_label.csv")

In [6]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 16026, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 16026 (delta 33), reused 41 (delta 25), pack-reused 15967
Receiving objects: 100% (16026/16026), 14.68 MiB | 29.83 MiB/s, done.
Resolving deltas: 100% (10999/10999), done.


In [7]:
%%bash
cd yolov5
pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.8/644.8 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.0 MB/s eta 0:00:00


In [9]:
df

,Unnamed: 0,imagepath,xmlpath,xmin,xmax,ymin,ymax,width,height,center_x,center_y,bb_width,bb_height
0,0,/content/drive/MyDrive/dataset/0073797c-a755-4...,/content/drive/MyDrive/dataset/0073797c-a755-4...,140,339,210,260,500,335,0.479000,0.701493,0.398000,0.149254
1,1,/content/drive/MyDrive/dataset/00b42b2c-f193-4...,/content/drive/MyDrive/dataset/00b42b2c-f193-4...,184,292,572,648,1280,853,0.185938,0.715123,0.084375,0.089097
2,2,/content/drive/MyDrive/dataset/018b52e6-e9a1-4...,/content/drive/MyDrive/dataset/018b52e6-e9a1-4...,327,399,202,227,660,280,0.550000,0.766071,0.109091,0.089286
3,3,/content/drive/MyDrive/dataset/03273806-bb1e-4...,/content/drive/MyDrive/dataset/03273806-bb1e-4...,185,374,290,339,588,476,0.475340,0.660714,0.321429,0.102941
4,4,/content/drive/MyDrive/dataset/0369b20e-b432-4...,/content/drive/MyDrive/dataset/0369b20e-b432-4...,335,453,313,347,480,480,0.820833,0.687500,0.245833,0.070833
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1036,1036,/content/drive/MyDrive/dataset/WB4.jpg,/content/drive/MyDrive/dataset/WB4.xml,119,191,268,292,272,363,0.569853,0.771350,0.264706,0.066116
1037,1037,/content/drive/MyDrive/dataset/WB5.jpg,/content/drive/MyDrive/dataset/WB5.xml,105,187,256,280,272,363,0.536765,0.738292,0.301471,0.066116
1038,1038,/content/drive/MyDrive/dataset/WB6.jpg,/content/drive/MyDrive/dataset/WB6.xml,108,181,247,266,272,363,0.531250,0.706612,0.268382,0.052342
1039,1039,/content/drive/MyDrive/dataset/WB8.jpg,/content/drive/MyDrive/dataset/WB8.xml,98,184,210,229,272,272,0.518382,0.806985,0.316176,0.069853


In [ ]:
df

,imagepath,xmlpath,xmin,xmax,ymin,ymax,width,height,center_x,center_y,bb_width,bb_height
0,/content/drive/MyDrive/dataset/0073797c-a755-4...,/content/drive/MyDrive/dataset/0073797c-a755-4...,140,339,210,260,500,335,0.479000,0.701493,0.398000,0.149254
1,/content/drive/MyDrive/dataset/00b42b2c-f193-4...,/content/drive/MyDrive/dataset/00b42b2c-f193-4...,184,292,572,648,1280,853,0.185938,0.715123,0.084375,0.089097
2,/content/drive/MyDrive/dataset/018b52e6-e9a1-4...,/content/drive/MyDrive/dataset/018b52e6-e9a1-4...,327,399,202,227,660,280,0.550000,0.766071,0.109091,0.089286
3,/content/drive/MyDrive/dataset/03273806-bb1e-4...,/content/drive/MyDrive/dataset/03273806-bb1e-4...,185,374,290,339,588,476,0.475340,0.660714,0.321429,0.102941
4,/content/drive/MyDrive/dataset/0369b20e-b432-4...,/content/drive/MyDrive/dataset/0369b20e-b432-4...,335,453,313,347,480,480,0.820833,0.687500,0.245833,0.070833
...,...,...,...,...,...,...,...,...,...,...,...,...
1036,/content/drive/MyDrive/dataset/WB4.jpg,/content/drive/MyDrive/dataset/WB4.xml,119,191,268,292,272,363,0.569853,0.771350,0.264706,0.066116
1037,/content/drive/MyDrive/dataset/WB5.jpg,/content/drive/MyDrive/dataset/WB5.xml,105,187,256,280,272,363,0.536765,0.738292,0.301471,0.066116
1038,/content/drive/MyDrive/dataset/WB6.jpg,/content/drive/MyDrive/dataset/WB6.xml,108,181,247,266,272,363,0.531250,0.706612,0.268382,0.052342
1039,/content/drive/MyDrive/dataset/WB8.jpg,/content/drive/MyDrive/dataset/WB8.xml,98,184,210,229,272,272,0.518382,0.806985,0.316176,0.069853


In [10]:
df_train = df.iloc[:800]
df_test = df.iloc[800:]

In [11]:
from shutil import copy

In [ ]:
train_folder = '/content/drive/MyDrive/data_images/train'

values = df_train[['imagepath','center_x','center_y','bb_width','bb_height']].values

for fname, x,y, w, h in values:
    image_name = os.path.split(fname)[-1]
    txt_name = os.path.splitext(image_name)[0]

    dst_image_path = os.path.join(train_folder,image_name)
    dst_label_file = os.path.join(train_folder,txt_name+'.txt')

    # copy each image into the folder
    copy(fname,dst_image_path)

    # generate .txt which has label info
    label_txt = f'0 {x} {y} {w} {h}'
    with open(dst_label_file,mode='w') as f:
        f.write(label_txt)

        f.close()

In [ ]:
test_folder = '/content/drive/MyDrive/data_images/test'

values = df_test[['imagepath','center_x','center_y','bb_width','bb_height']].values
for fname, x,y, w, h in values:
    image_name = os.path.split(fname)[-1]
    txt_name = os.path.splitext(image_name)[0]

    dst_image_path = os.path.join(test_folder,image_name)
    dst_label_file = os.path.join(test_folder,txt_name+'.txt')

    # copy each image into the folder
    copy(fname,dst_image_path)

    # generate .txt which has label info
    label_txt = f'0 {x} {y} {w} {h}'
    with open(dst_label_file,mode='w') as f:
        f.write(label_txt)

        f.close()

In [12]:
os.chdir('/content/yolov5')

In [13]:
ls


benchmarks.py    data/       LICENSE          requirements.txt  tutorial.ipynb
CITATION.cff     detect.py   models/          segment/          utils/
classify/        export.py   README.md        setup.cfg         val.py
CONTRIBUTING.md  hubconf.py  README.zh-CN.md  train.py


In [14]:
with open("/content/yolov5/data.yaml" , 'w') as f:
  f.write("train : /content/drive/MyDrive/data_images/train\nval : /content/drive/MyDrive/data_images/test\nnc: 1\nnames: ['license_plate']")

In [16]:
ls

benchmarks.py  CONTRIBUTING.md  detect.py   LICENSE    README.zh-CN.md   setup.cfg       utils/
CITATION.cff   data/            export.py   models/    requirements.txt  train.py        val.py
classify/      data.yaml        hubconf.py  README.md  segment/          tutorial.ipynb


In [17]:
!python train.py --data data.yaml --batch-size 8 --name Model4 --epochs 100 --weights yolov5s.pt

2023-10-28 03:35:20.517755: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-28 03:35:20.517864: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-28 03:35:20.517913: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5s.pt, cfg=, data=data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size=8, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=ru

In [ ]:
os.chdir('/content/yolov5')

In [19]:
!python export.py --weights runs/train/Model4/weights/best.pt --include onnx --simplify

export: data=data/coco128.yaml, weights=['runs/train/Model4/weights/best.pt'], imgsz=[640, 640], batch_size=1, device=cpu, half=False, inplace=False, keras=False, optimize=False, int8=False, dynamic=False, simplify=True, opset=17, verbose=False, workspace=4, nms=False, agnostic_nms=False, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, include=['onnx']
YOLOv5 🚀 v7.0-230-g53efd07 Python-3.10.12 torch-2.1.0+cu118 CPU

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs

PyTorch: starting from runs/train/Model4/weights/best.pt with output shape (1, 25200, 6) (13.7 MB)
requirements: Ultralytics requirement ['onnx>=1.12.0'] not found, attempting AutoUpdate...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 123.8 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 9.2s, installed 1 package: ['onnx>=1.12.0']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.15.0.